In [ ]:
import subprocess
import shlex
import re

import numpy as np

from IPython.display import display, Markdown
from tqdm import tqdm

In [ ]:
CC_PATTERN = re.compile(r"(?<!^)(?=[A-Z])")

# PractRand (Practically Random) - statistical tests for RNGs

In [ ]:
def test_randomness_practrand(
    mode, seeds=[42], limit='1GB',
):
    outputs = []
    
    for seed in tqdm(seeds):
        generator = subprocess.Popen(shlex.split(
            f"cargo run --release --quiet -- --seed {seed} {CC_PATTERN.sub('-', mode).lower()}"
        ), stdout=subprocess.PIPE)
        practrand = subprocess.run(
            shlex.split(f"./practrand stdin64 -tlmax {limit}"), stdin=generator.stdout,
            check=True, capture_output=True, text=True
        )

        outputs.append(practrand.stdout)
        
    most_suspect = None
    
    for output in outputs:
        failures = output.count("FAIL")
        suspicions = output.count("SUSPICIOUS") * 2 + output.count("suspicious")
        unusuals = output.count("unusual")
        
        suspicion = failures * 10 + suspicions * 2 + unusuals
        
        if (most_suspect is None) or (suspicion > most_suspect[0]):
            most_suspect = (suspicion, output)

    if "FAIL" in most_suspect[1]:
        display(Markdown(f"## <span style='color:purple'><u>{mode}</u></span>"))
    elif "SUSPICIOUS" in most_suspect[1]:
        display(Markdown(f"## <span style='color:red'><u>{mode}</u></span>"))
    elif "suspicious" in most_suspect[1]:
        display(Markdown(f"## <span style='color:orange'>*{mode}*</span>"))
    else:
        display(Markdown(f"## <span style='color:green'>{mode}</span>"))
        
    display(Markdown("#### Test output (most suspect):"))
    display(Markdown(">" + most_suspect[1].replace('\n', '\n>')))

    display(Markdown("#### Parameters:"))
    display(Markdown(f"* seeds: {seeds}\n* limit: {limit}"))

In [ ]:
for mode in ["Monolithic", "Independent", "IndependentSimulation", "IndependentSimulationNoDispersal"]:
    seeds = np.random.randint(0, np.iinfo("uint64").max, dtype="uint64", size=10)
    
    test_randomness_practrand(mode, seeds=seeds, limit='512GB')